# APIM ❤️ OpenAI

## GPT-4o 推論ラボ
![flow](../../images/GPT-4o-inferencing.gif)

新しいGPT-4oモデルを試すためのプレイグラウンドです。GPT-4o（"o"は"omni"の略）は、テキスト、音声、ビデオの入力を組み合わせて処理し、テキスト、音声、画像形式で出力を生成するように設計されています。

### 目次
- [0️⃣ ノートブック変数の初期化](#0)
- [1️⃣ Azureリソースグループの作成](#1)
- [2️⃣ 🦾 Bicepを使用したデプロイの作成](#2)
- [3️⃣ デプロイの出力を取得](#3)
- [🧪 Azure OpenAI Python SDKを使用したAPIのテスト](#sdk1)
- [🧪 画像処理](#sdk2)
- [🧪 Base64画像処理](#sdk3)
- [🗑️ リソースのクリーンアップ](#clean)

### 前提条件
- [Python 3.8以降のバージョン](https://www.python.org/)がインストールされていること
- [Pandasライブラリ](https://pandas.pydata.org/)がインストールされていること
- [VS Code](https://code.visualstudio.com/)がインストールされ、[Jupyterノートブック拡張機能](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)が有効になっていること
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)がインストールされていること
- Contributor権限を持つ[Azureサブスクリプション](https://azure.microsoft.com/en-us/free/)があること
- [Azure OpenAIへのアクセス](https://aka.ms/oai/access)が許可されていること、またはモックサービスを有効にすること
- [Azure CLIでAzureにサインイン](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)していること

<a id='0'></a>
### 0️⃣ ノートブック変数の初期化

- リソースはサブスクリプションIDに基づいた一意の文字列でサフィックスされます
- ```mock_webapps``` 変数は、モック機能のためにデプロイされたWebアプリのリストを設定します。モックサービスでOpenAIの動作をシミュレートするために、```openai_resources``` リストをクリアします。
- ロケーションパラメータは、[Azureリージョンごとの製品の利用可能性](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)に基づいて調整してください。
- OpenAIモデルとバージョンは、[リージョンごとの利用可能性](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)に基づいて調整してください。

In [ ]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"
openai_resources = [ {"name": "openai1", "location": "westus3"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_resources_sku = "S0"
openai_model_name = "gpt-4o"
openai_model_version = "2024-05-13"
openai_deployment_name = "gpt-4o"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ {"name": "openaimock1", "endpoint": "https://openaimock1.azurewebsites.net"}, {"name": "openaimock2", "endpoint": "https://openaimock2.azurewebsites.net"} ]

log_analytics_name = "workspace"
app_insights_name = 'insights'


<a id='1'></a>
### 1️⃣ Azureリソースグループの作成
このラボでデプロイされるすべてのリソースは、指定されたリソースグループに作成されます。既存のリソースグループを使用する場合は、このステップをスキップしてください。

In [ ]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

<a id='2'></a>
### 2️⃣ 🦾 Bicepを使用したデプロイの作成

このラボでは、[Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep)を使用して、デプロイされるすべてのリソースを宣言的に定義します。異なる構成を試すために、パラメータや[main.bicep](main.bicep)を直接変更してください。

In [ ]:
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")
elif len(mock_webapps) > 0:
    backend_id = mock_backend_pool if len(mock_backend_pool) > 1 else mock_webapps[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name }
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


<a id='3'></a>
### 3️⃣ デプロイの出力を取得

テストの準備が整う前に、ゲートウェイURLとサブスクリプションを取得する必要があります。

In [ ]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)



<a id='requsts'></a>
### 🧪 直接HTTPコールを使用したAPIのテスト
Requestsは、ここで生のAPIリクエストを行い、レスポンスを検査するために使用される、エレガントでシンプルなPython用のHTTPライブラリです。

In [ ]:
import time
runs = 1
sleep_time_ms = 1000

url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

for i in range(runs):
    print("▶️ Run: ", i+1)
    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}
    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)
    print("status code: ", response.status_code)
    print("headers ", response.headers)
    print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)
    time.sleep(sleep_time_ms/1000)


<a id='sdk1'></a>
### 🧪 Azure OpenAI Python SDKを使用したAPIのテスト

In [ ]:
import time
from openai import AzureOpenAI
runs = 1
sleep_time_ms = 1000

for i in range(runs):
    print("▶️ Run: ", i+1)
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]
    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key,
        api_version=openai_api_version
    )
    response = client.chat.completions.create(model=openai_model_name, messages=messages)
    print("💬 ", response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)


<a id='sdk2'></a>
### 🧪 画像処理

GPT-4oは画像を直接処理し、画像に基づいてインテリジェントなアクションを実行できます。

以下の例は[OpenAI Cookbook](https://cookbook.openai.com/examples/gpt4o/introduction_to_gpt4o)から適応されています。

#### 入力画像:
![image](https://upload.wikimedia.org/wikipedia/commons/e/e2/The_Algebra_of_Mohammed_Ben_Musa_-_page_82b.png)

In [ ]:
from IPython.display import display, Markdown
response = client.chat.completions.create(
    model=openai_model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that responds in Markdown. Help me with my math homework!"},
        {"role": "user", "content": [
            {"type": "text", "text": "What's the area of the triangle?"},
            {"type": "image_url", "image_url": {
                "url": "https://upload.wikimedia.org/wikipedia/commons/e/e2/The_Algebra_of_Mohammed_Ben_Musa_-_page_82b.png"}
            }
        ]}
    ],
    temperature=0.0,
)

display(Markdown(response.choices[0].message.content))


<a id='sdk3'></a>
### 🧪 Base64画像処理

#### 入力画像:
<img src="meme.png" width="600px">

In [ ]:
from IPython.display import display, Markdown, HTML
import base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
base64_image = encode_image("meme.png")
response = client.chat.completions.create(
    model=openai_model_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that helps to interprete memes!"},
        {"role": "user", "content": [
            {"type": "text", "text": "What's funny about this picture?"},
            {"type": "image_url", "image_url": {
                "url": f"data:image/png;base64,{base64_image}"}
            }
        ]}
    ],
    temperature=0.0,
)
HTML('<font size="5">' + response.choices[0].message.content + '</font>')

<a id='clean'></a>
### 🗑️ リソースのクリーンアップ

ラボが終了したら、追加の料金を避け、Azureサブスクリプションを整理するために、デプロイしたすべてのリソースをAzureから削除する必要があります。
そのためには、[リソースのクリーンアップノートブック](clean-up-resources.ipynb)を使用してください。